# GENERAL DOMAIN

In [6]:
import transformers
import torch
from datasets import load_dataset
import json
from transformers import AutoModelForCausalLM, AutoTokenizer

# Huggingface token
token = ''

# Cache path
local_cache_path = ''

# Saved json file path
saved_json_file = ''


model_name = "meta-llama/Meta-Llama-3-8B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name,token=token,)
tokenizer.pad_token = 'None'

In [3]:
local_dataset = load_dataset('togethercomputer/RedPajama-Data-1T','arxiv',split = 'train',streaming=True,)
shuffled_dataset = local_dataset.shuffle(seed = 42, buffer_size = 10000)

The repository for togethercomputer/RedPajama-Data-1T contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/togethercomputer/RedPajama-Data-1T.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N]  y


In [10]:
token = 0
m = 0

# token needed for general domain
token_needed = 100000#14497666456

with open(saved_json_file,'a') as fw:
    for i in shuffled_dataset:

        m+=1
        if token <= token_needed: 
            text = i['text']
            #token += i['token_count']
            tokenized_batch = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
            token+=sum(sum(tokenized_batch['attention_mask'].T))
            data = {'text': f'{text}'}
            #print(token)
            fw.write(json.dumps(data) + '\n')

            if m%1000000 == 0:
                print(token/token_needed)

        else:
            print('done')
            break

tensor(22060)
tensor(54434)
tensor(61043)
tensor(109941)
done


# BIO DOMAIN

In [14]:
from datasets import load_dataset
data1 = 'epfl-llm/guidelines'
data2 = 'health360/Healix-Shot'

local_dataset1 = load_dataset(data1,cache_dir = local_cache_path)['train']
local_dataset2 = load_dataset(data2,cache_dir = local_cache_path)['train']

Resolving data files:   0%|          | 0/294 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/288 [00:00<?, ?it/s]

In [17]:
#data1---epfl-llm/guidelines
#data2---health360/Healix-Shot

import json
with open(saved_json_file,'a') as f:
    for i in local_dataset1['train']:
        
        text = i['clean_text']  # for data1, use i['clean_text']; for data2, use i['text']
        
        data = {'text': f'{text}'}
        f.write(json.dumps(data) + '\n')

In [ ]:
# For text with jsonl format.
# text3---the-pile-pubmed-central-refine-result.jsonl
data3_path = ''

import json
with open(data3_path, 'r') as file, open(saved_json_file,'a') as fw:
    k = 0
    for line in file:
        if k%50000 == 0:
            print(1)
        k+=1
        data_line = json.loads(line)
        
        text = data_line['text']
        data = {'text': f'{text}'}
        fw.write(json.dumps(data) + '\n')